In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!pwd

/home/jovyan/ds_nudge_up/playground


In [3]:
import sys
sys.path += ["../"]

import os
import src.utils as utl
rasterio_config = utl.load_yaml_config("rasterioc_config.yaml")
os.environ.update(rasterio_config)

from src.featurize import *
from src.featurize.nn_forward_pass import featurize
import torch
import numpy as np
import pandas as pd
from pathlib import Path

## Load files

In [4]:
featurization_params = utl.load_yaml_config("featurisation.yaml")
satellite_config = utl.load_yaml_config("satellite_config.yaml")
satellite_config = satellite_config[featurization_params['satellite_search_params']['satellite_name']]
data_catalog = utl.load_yaml_config('data_catalog.yaml')

In [5]:
points_gdf = utl.load_points_gdf(**data_catalog['request_points'])

## Get STAC items

In [6]:
# %%time
points_gdf_with_stac = parallel_fetch_stac_items(
    points_gdf, 
    featurization_params['dask']['n_partitions'], 
    featurization_params['satellite_search_params']
)
# points_gdf_with_stac = points_gdf_with_stac.sort_index().reset_index()

In [7]:
# points_gdf_with_stac.stac_item.iloc[0]

## Create features

In [8]:
data_loader = create_data_loader(points_gdf_with_stac, satellite_config, featurization_params['batch_size'])

In [9]:
device = torch.device(featurization_params['device'])

In [10]:
model = RCF(
    featurization_params['num_features'], 
    featurization_params['kernel_size'], 
    len(satellite_config['bands'])
).eval().to(device)

In [11]:
X_features = create_features(
    data_loader, 
    featurization_params['num_features'], 
    len(points_gdf_with_stac), 
    model, 
    device, 
    satellite_config['min_image_edge']
)

  0%|          | 0/99897 [00:00<?, ?it/s]

ValueError: zero-size array to reduction operation minimum which has no identity

## Save to file

In [ ]:
ROOT = Path.cwd().parent
output_folder_path = ROOT/"data/01_preprocessed/mosaiks_features/"
os.makedirs(output_folder_path, exist_ok=True)

In [ ]:
X_features_df = pd.DataFrame(X_features, columns=[str(c) for c in range(X_features.shape[1])])
gdf = pd.concat([points_gdf_with_stac.drop("stac_item", axis=1), X_features_df], axis=1)

In [ ]:
filename = "mosaiks_points_latest"
gdf.to_parquet(output_folder_path / f"{filename}.parquet.gzip", compression="gzip")